<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Projects/blob/main/3D_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import torch
import torchvision
import torchsummary
import numpy as np
import pandas as pd
from PIL import Image
from sklearn import metrics
from matplotlib import pyplot as plt
from IPython.display import clear_output
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive'
!kaggle datasets download -d awsaf49/rsna-atd-512x512-png-v2-dataset
!unzip \*.zip && rm *.zip
clear_output()